In [2]:
print("All ok.....")

All ok.....


In [3]:
import os
import json

### Extracting Conversation from All_Conversations folder

In [15]:
import yaml
import os

directory = "All_Conversations"

conversations = {}
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(os.path.join(directory, filename), 'r') as file:
            conversations[filename] = json.load(file)

### Now conversations will contain all the data we need for further analysis.


### List of profane words (Source -> Google)

In [9]:
profane_words = [
    "arse", "arsehead", "arsehole", "ass", "ass hole", "asshole", 
    "bastard", "bitch", "bloody", "bollocks", "brotherfucker", "bugger", 
    "bullshit", "child-fucker", "Christ on a bike", "Christ on a cracker", 
    "cock", "cocksucker", "crap", "cunt", "dammit", "damn", "damned", 
    "damn it", "dick", "dick-head", "dickhead", "dumb ass", "dumb-ass", 
    "dumbass", "dyke", "faggot", "father-fucker", "fatherfucker", "fuck", 
    "fucker", "fucking", "god dammit", "goddammit", "God damn", "god damn", 
    "goddamn", "Goddamn", "goddamned", "goddamnit", "godsdamn", "hell", 
    "holy shit", "horseshit", "in shit", "jackarse", "jack-ass", "jackass", 
    "Jesus Christ", "Jesus fuck", "Jesus Harold Christ", "Jesus H. Christ", 
    "Jesus, Mary and Joseph", "Jesus wept", "kike", "mother fucker", 
    "mother-fucker", "motherfucker", "nigga", "nigra", "pigfucker", "piss", 
    "prick", "pussy", "shit", "shit ass", "shite", "sibling fucker", 
    "sisterfuck", "sisterfucker", "slut", "son of a bitch", "son of a whore", 
    "spastic", "sweet Jesus", "twat", "wanker"
]

# Question-1. Profanity Detection

### a. Identify all the call ids where collection agents have used profane language.
### b. Identify all the call ids where borrowers have used profane language.

## ->  Using Regex

In [89]:
import re
profane_calls_re = {"Agent": [], "Customer": []}
def detect_profanity(conversations, profane_words):
    for call_id, conversation in conversations.items(): # Looping in conversations
        for entry in conversation:
            speaker = entry['speaker']
            text = entry['text']

            if any(re.search(rf"\b{word}\b", text, re.IGNORECASE) for word in profane_words): # Checking for profane in profane_words
                if speaker == 'Agent': # If Agent use bad words then appending call_id in Agent
                    profane_calls_re["Agent"].append(call_id)
                elif speaker == 'Customer': # Else in Customer
                    profane_calls_re["Customer"].append(call_id)
    
    return profane_calls

profane_calls = detect_profanity(conversations, profane_words)


In [98]:
# Removing Duplicate Call IDs
profane_calls_re["Agent"] = list(set(profane_calls_re["Agent"]))
profane_calls_re["Customer"] = list(set(profane_calls_re["Customer"]))

print("Calls with profane words by Agent:", profane_calls_re["Agent"])
print(len(profane_calls_re["Agent"]))
print("Calls with profane words by Customer:", profane_calls_re["Customer"])
print(len(profane_calls_re["Customer"]))


Calls with profane words by Agent: ['2f279983-ea96-4c10-a09d-4b607de21a38.json', 'c67274cc-d920-467a-80cc-476c9dd280d4.json', 'e0a8f9d2-92d6-4093-bb97-82182c6e850f.json', '966d60ed-9111-4e44-876d-6fcc3d78773e.json', '87e28dae-2c70-4122-9452-6ec82164fab2.json', '52ddb4a0-0599-4e18-a961-ac4b67da8d5e.json', 'f5f05257-aaa0-4194-b84c-36339c4e0659.json', '50abe040-847c-47e1-962f-a00d7ff2cfe4.json', 'bf973b43-76ec-41a6-af68-a3b4bc4b69b3.json', '4f05aaff-585f-485f-90a1-60d027ec6e46.json', '928acb70-fa80-4532-a0c0-2234bf236e17.json', '29489f77-b8e0-462c-9286-9216d59890f1.json', '9505f0e7-5404-4f50-a497-35a9e899197c.json', '23091fb3-8e07-4d4c-8575-227e3467e73a.json', '89d28c5a-f0da-41e9-a93e-0a749c20189c.json', '214c92cc-3141-404d-8224-c1b05acc345c.json', '04bec80f-8614-484b-8ba2-831ff9dd03ef.json', '5e231fa2-03d8-4e70-9918-da45193472a9.json', '9a6cb7d2-c697-43a0-8cad-4194dfb44c92.json']
19
Calls with profane words by Customer: ['584fd5cd-adaa-4315-bf67-ad56d02d135e.json', 'd6a3e0da-171f-4744-ba

## ->  Using Machine learning approach (LLM)

#### initializing huggingface fine-tuned model 

In [91]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [92]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use mps:0


In [93]:
def detect_profanity_hf(text):
    result=classifier(text)
    if result[0]['score'] >= 0.5:
        return "profane"
    else:
        return "non-profane"

#### detect_profanity_in_conversations_hf will take around 1-2 min for final result

In [ ]:
profane_calls_LLM = {"Agent": [], "Customer": []}
def detect_profanity_in_conversations_hf(conversations):
    for call_id, conversation in conversations.items():  # Looping in conversations
        for entry in conversation:
            speaker = entry['speaker']
            text = entry['text']

            result = detect_profanity_hf(text)
            
            if result == "profane": # Checking for profane in profane_words
                if speaker == 'Agent': # If Agent use bad words then appending call_id in Agent
                    profane_calls_LLM["Agent"].append(call_id) 
                elif speaker == 'Customer': # Else in Customer
                    profane_calls_LLM["Customer"].append(call_id)
    
    return profane_calls

profane_calls = detect_profanity_in_conversations_hf(conversations)


In [97]:
# Removing Duplicate Call IDs
profane_calls_LLM["Agent"] = list(set(profane_calls_LLM["Agent"]))
profane_calls_LLM["Customer"] = list(set(profane_calls_LLM["Customer"]))

print("Calls with profane words by Agent:", profane_calls_LLM["Agent"])
print(len(profane_calls_LLM["Agent"]))
print("Calls with profane words by Customer:", profane_calls_LLM["Customer"])
print(len(profane_calls_LLM["Customer"]))


Calls with profane words by Agent: ['f5f05257-aaa0-4194-b84c-36339c4e0659.json', '23091fb3-8e07-4d4c-8575-227e3467e73a.json', '50abe040-847c-47e1-962f-a00d7ff2cfe4.json', '2f279983-ea96-4c10-a09d-4b607de21a38.json', 'bf973b43-76ec-41a6-af68-a3b4bc4b69b3.json', '9505f0e7-5404-4f50-a497-35a9e899197c.json', '89d28c5a-f0da-41e9-a93e-0a749c20189c.json', 'c67274cc-d920-467a-80cc-476c9dd280d4.json', '29489f77-b8e0-462c-9286-9216d59890f1.json', 'e0a8f9d2-92d6-4093-bb97-82182c6e850f.json', '214c92cc-3141-404d-8224-c1b05acc345c.json', '04bec80f-8614-484b-8ba2-831ff9dd03ef.json', '4f05aaff-585f-485f-90a1-60d027ec6e46.json', '966d60ed-9111-4e44-876d-6fcc3d78773e.json', '5e231fa2-03d8-4e70-9918-da45193472a9.json', '9a6cb7d2-c697-43a0-8cad-4194dfb44c92.json', '87e28dae-2c70-4122-9452-6ec82164fab2.json', '928acb70-fa80-4532-a0c0-2234bf236e17.json']
18
Calls with profane words by Customer: ['584fd5cd-adaa-4315-bf67-ad56d02d135e.json', 'd6a3e0da-171f-4744-ba2b-7298f2604e7f.json', '20690906-b8d4-40c5-84

## -> Comparative Analysis (Question 1)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Combining Agent and Customer call IDs for overall analysis
regex_combined_set = set(profane_calls_re["Agent"]).union(set(profane_calls_re["Customer"]))
llm_combined_set = set(profane_calls_LLM["Agent"]).union(set(profane_calls_LLM["Customer"]))

# Assume LLM is correct (evaluate Regex against LLM)
TP_regex_vs_llm = len(regex_combined_set.intersection(llm_combined_set))  # True positives
FP_regex_vs_llm = len(regex_combined_set.difference(llm_combined_set))   # False positives
FN_regex_vs_llm = len(llm_combined_set.difference(regex_combined_set))   # False negatives

precision_regex_vs_llm = TP_regex_vs_llm / (TP_regex_vs_llm + FP_regex_vs_llm) if (TP_regex_vs_llm + FP_regex_vs_llm) > 0 else 0
recall_regex_vs_llm = TP_regex_vs_llm / (TP_regex_vs_llm + FN_regex_vs_llm) if (TP_regex_vs_llm + FN_regex_vs_llm) > 0 else 0
f1_regex_vs_llm = 2 * (precision_regex_vs_llm * recall_regex_vs_llm) / (precision_regex_vs_llm + recall_regex_vs_llm) if (precision_regex_vs_llm + recall_regex_vs_llm) > 0 else 0

# Assume Regex is correct (evaluate LLM against Regex)
TP_llm_vs_regex = len(llm_combined_set.intersection(regex_combined_set))  # True positives
FP_llm_vs_regex = len(llm_combined_set.difference(regex_combined_set))   # False positives
FN_llm_vs_regex = len(regex_combined_set.difference(llm_combined_set))   # False negatives

precision_llm_vs_regex = TP_llm_vs_regex / (TP_llm_vs_regex + FP_llm_vs_regex) if (TP_llm_vs_regex + FP_llm_vs_regex) > 0 else 0
recall_llm_vs_regex = TP_llm_vs_regex / (TP_llm_vs_regex + FN_llm_vs_regex) if (TP_llm_vs_regex + FN_llm_vs_regex) > 0 else 0
f1_llm_vs_regex = 2 * (precision_llm_vs_regex * recall_llm_vs_regex) / (precision_llm_vs_regex + recall_llm_vs_regex) if (precision_llm_vs_regex + recall_llm_vs_regex) > 0 else 0

# Print metrics for both assumptions
print("Evaluation when assuming LLM is correct:")
print(f"True Positives: {TP_regex_vs_llm}")
print(f"False Positives (Regex): {FP_regex_vs_llm}")
print(f"False Negatives (Regex): {FN_regex_vs_llm}")
print(f"Precision (Regex): {precision_regex_vs_llm:.4f}")
print(f"Recall (Regex): {recall_regex_vs_llm:.4f}")
print(f"F1-Score (Regex): {f1_regex_vs_llm:.4f}")

print("\nEvaluation when assuming Regex is correct:")
print(f"True Positives: {TP_llm_vs_regex}")
print(f"False Positives (LLM): {FP_llm_vs_regex}")
print(f"False Negatives (LLM): {FN_llm_vs_regex}")
print(f"Precision (LLM): {precision_llm_vs_regex:.4f}")
print(f"Recall (LLM): {recall_llm_vs_regex:.4f}")
print(f"F1-Score (LLM): {f1_llm_vs_regex:.4f}")

# Determine the better approach based on the results
if f1_regex_vs_llm > f1_llm_vs_regex:
    print("\nBased on the evaluation, the Regex approach performs better.")
elif f1_regex_vs_llm < f1_llm_vs_regex:
    print("\nBased on the evaluation, the LLM approach performs better.")
else:
    print("\nBoth approaches perform equally well based on the evaluation.")


Evaluation when assuming LLM is correct:
True Positives: 38
False Positives (Regex): 1
False Negatives (Regex): 0
Precision (Regex): 0.9744
Recall (Regex): 1.0000
F1-Score (Regex): 0.9870

Evaluation when assuming Regex is correct:
True Positives: 38
False Positives (LLM): 0
False Negatives (LLM): 1
Precision (LLM): 1.0000
Recall (LLM): 0.9744
F1-Score (LLM): 0.9870

Both approaches perform equally well based on the evaluation.


# Question-2. Privacy and Compliance Analysis

### a. Identify all the call ids where agents have shared sensitive information (balance or account details) without the identity verification(i.e. without verification of date of birth or address or Social Security Number).

## -> Using Regex

In [147]:
import re

# Define regex patterns
sensitive_pattern = re.compile(r"\b(balance|amount owed|total due|payment|account number|credit card)\b", re.IGNORECASE)
verification_pattern = re.compile(r"\b(date of birth|DOB|address|street|zip code|SSN|social security number)\b", re.IGNORECASE)

violating_call_ids_regex = []

for call_id, conversation in conversations.items():
    agent_statements = [utt["text"] for utt in conversation if utt["speaker"] == "Agent"]
    verified = False
    violation = False

    for statement in agent_statements:
        if verification_pattern.search(statement):
            verified = True  # Found verification keyword
        if sensitive_pattern.search(statement) and not verified:
            violation = True  # Found sensitive information without prior verification
            break
    
    if violation:
        violating_call_ids_regex.append(call_id)

print("Call IDs with sensitive information violations:", violating_call_ids_regex)


Call IDs with sensitive information violations: ['19169ec6-213f-48e9-8ec2-c24ee2e6eb20.json', '80e5fe02-ec03-4ca5-902a-fdf38e6b7b8a.json', 'acec9db1-1209-4f98-881e-6d413e1aba2d.json', 'd7bbea61-d739-43fb-a198-ced1b59f9491.json', '8372d771-4271-47b1-82e7-9cb3562fb380.json', '39c3e29c-7b7c-43d2-999b-eb473ec31de3.json', '60e92cb0-536d-4b03-b778-81dcbef4ea0e.json', '38231782-a07c-4fbb-a036-723dfdfede1e.json', '00be25b0-458f-4cbf-ae86-ae2ec1f7fba4.json', '40b40ec2-8ce2-47a8-b9d4-621285d7b484.json', 'dbd3e2fd-4a8f-4eb3-aa6f-befcc150bfd8.json', '1190bab7-d82f-4259-bb55-9617dff7da07.json', 'ddaea5f2-20b8-439a-a0be-ac92bb606e7a.json', '9e936113-46a1-4d70-9cc5-876b1c801602.json', 'ecf48f31-9a97-4042-bd79-0c24e6ecb4af.json', '708e7950-1be4-47a7-9c82-e03c88f5a1d0.json', '742147aa-a2df-4429-859e-05b4a7875563.json', 'ee078707-82d8-4937-942a-0d1dce58dac5.json', 'b6eb2096-6102-431d-b0df-4a87310d46c4.json', '5c0bf9ad-669f-4a26-8c08-4af6f9698cf0.json', 'bf973b43-76ec-41a6-af68-a3b4bc4b69b3.json', '87925

## -> Using LLM (Cerebras API for fast inferencing)

In [141]:
from langchain_cerebras import ChatCerebras
cerebras_llm = ChatCerebras(model="llama-3.3-70b", api_key="csk-v3e6jjknn2pmvk5t2mx5jncyxvj6d32he6phyxcj886vrcte")
def check_compliance_with_cerebras(conversation):
    # Define the prompt for the analysis
    prompt = f"""
You are a compliance auditor. Analyze the following conversation between an Agent and a Customer to determine if the Agent shared sensitive information (like outstanding balance amount , account number, or credit card details) **without first verifying the Customer's identity**. return "Violation" if the Agent violated the compliance policy, and "Good" if the Agent followed the correct procedure.

The following checks must be strictly applied:
1. Identity verification must include confirmation of at least one of the following:
- Date of birth (DOB)
- Address
- Social Security Number (SSN)
2. If the Agent provides sensitive information without any of these verifications, return "Violation".
3. If the Agent performs verification before sharing sensitive information, return "Good".
4. Do not include any explanation, comments, or additional text. Only return "Violation" or "Good".

Conversation:
{conversation}

Strict Evaluation:
"""

    # Send the prompt to the Cerebras model
    response = cerebras_llm.invoke(prompt)
    return response

In [144]:
# Example usage on conversations
violating_call_ids = []
for call_id, conversation in conversations.items():
    # Combine conversation into a single string
    result = check_compliance_with_cerebras(conversation)
    result = result.content
    if "Violation" in result:
        violating_call_ids.append(call_id)

print("Call IDs with sensitive information violations:", violating_call_ids)


Call IDs with sensitive information violations: ['19169ec6-213f-48e9-8ec2-c24ee2e6eb20.json', '80e5fe02-ec03-4ca5-902a-fdf38e6b7b8a.json', 'acec9db1-1209-4f98-881e-6d413e1aba2d.json', 'd7bbea61-d739-43fb-a198-ced1b59f9491.json', '8372d771-4271-47b1-82e7-9cb3562fb380.json', '39c3e29c-7b7c-43d2-999b-eb473ec31de3.json', '60e92cb0-536d-4b03-b778-81dcbef4ea0e.json', 'd071bb49-40f8-4bae-8c6d-cfc0d4a011d5.json', '38231782-a07c-4fbb-a036-723dfdfede1e.json', '00be25b0-458f-4cbf-ae86-ae2ec1f7fba4.json', '40b40ec2-8ce2-47a8-b9d4-621285d7b484.json', 'f152d094-a169-4486-b9af-a66a7716a608.json', 'dbd3e2fd-4a8f-4eb3-aa6f-befcc150bfd8.json', '1190bab7-d82f-4259-bb55-9617dff7da07.json', 'ddaea5f2-20b8-439a-a0be-ac92bb606e7a.json', 'b70866a2-2f46-4784-992b-74d6dc60806e.json', '9e936113-46a1-4d70-9cc5-876b1c801602.json', 'ecf48f31-9a97-4042-bd79-0c24e6ecb4af.json', '708e7950-1be4-47a7-9c82-e03c88f5a1d0.json', 'a73d777f-368a-46b5-9d59-0aa5829b4427.json', '742147aa-a2df-4429-859e-05b4a7875563.json', 'ee078

- > Better but Time taking approach (around 8 min)

## -> Comparative Analysis (Question 2)

In [152]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Convert the lists to sets for efficient operations
regex_set = set(violating_call_ids_regex)
llm_set = set(violating_call_ids)

# Assume LLM is correct (evaluate Regex against LLM)
TP_regex_vs_llm = len(regex_set.intersection(llm_set))  # True positives
FP_regex_vs_llm = len(regex_set.difference(llm_set))    # False positives
FN_regex_vs_llm = len(llm_set.difference(regex_set))    # False negatives

precision_regex_vs_llm = TP_regex_vs_llm / (TP_regex_vs_llm + FP_regex_vs_llm) if (TP_regex_vs_llm + FP_regex_vs_llm) > 0 else 0
recall_regex_vs_llm = TP_regex_vs_llm / (TP_regex_vs_llm + FN_regex_vs_llm) if (TP_regex_vs_llm + FN_regex_vs_llm) > 0 else 0
f1_regex_vs_llm = 2 * (precision_regex_vs_llm * recall_regex_vs_llm) / (precision_regex_vs_llm + recall_regex_vs_llm) if (precision_regex_vs_llm + recall_regex_vs_llm) > 0 else 0

# Assume Regex is correct (evaluate LLM against Regex)
TP_llm_vs_regex = len(llm_set.intersection(regex_set))  # True positives
FP_llm_vs_regex = len(llm_set.difference(regex_set))    # False positives
FN_llm_vs_regex = len(regex_set.difference(llm_set))    # False negatives

precision_llm_vs_regex = TP_llm_vs_regex / (TP_llm_vs_regex + FP_llm_vs_regex) if (TP_llm_vs_regex + FP_llm_vs_regex) > 0 else 0
recall_llm_vs_regex = TP_llm_vs_regex / (TP_llm_vs_regex + FN_llm_vs_regex) if (TP_llm_vs_regex + FN_llm_vs_regex) > 0 else 0
f1_llm_vs_regex = 2 * (precision_llm_vs_regex * recall_llm_vs_regex) / (precision_llm_vs_regex + recall_llm_vs_regex) if (precision_llm_vs_regex + recall_llm_vs_regex) > 0 else 0

# Print metrics for both assumptions
print("Evaluation when assuming LLM is correct:")
print(f"True Positives: {TP_regex_vs_llm}")
print(f"False Positives (Regex): {FP_regex_vs_llm}")
print(f"False Negatives (Regex): {FN_regex_vs_llm}")
print(f"Precision (Regex): {precision_regex_vs_llm:.4f}")
print(f"Recall (Regex): {recall_regex_vs_llm:.4f}")
print(f"F1-Score (Regex): {f1_regex_vs_llm:.4f}")

print("\nEvaluation when assuming Regex is correct:")
print(f"True Positives: {TP_llm_vs_regex}")
print(f"False Positives (LLM): {FP_llm_vs_regex}")
print(f"False Negatives (LLM): {FN_llm_vs_regex}")
print(f"Precision (LLM): {precision_llm_vs_regex:.4f}")
print(f"Recall (LLM): {recall_llm_vs_regex:.4f}")
print(f"F1-Score (LLM): {f1_llm_vs_regex:.4f}")

# Determine the better approach based on the results
if f1_regex_vs_llm > f1_llm_vs_regex:
    print("\nBased on the evaluation, the Regex approach performs better.")
elif f1_regex_vs_llm < f1_llm_vs_regex:
    print("\nBased on the evaluation, the LLM approach performs better.")
else:
    print("\nBoth approaches perform equally well based on the evaluation.")


Evaluation when assuming LLM is correct:
True Positives: 73
False Positives (Regex): 0
False Negatives (Regex): 10
Precision (Regex): 1.0000
Recall (Regex): 0.8795
F1-Score (Regex): 0.9359

Evaluation when assuming Regex is correct:
True Positives: 73
False Positives (LLM): 10
False Negatives (LLM): 0
Precision (LLM): 0.8795
Recall (LLM): 1.0000
F1-Score (LLM): 0.9359

Both approaches perform equally well based on the evaluation.


- > **Note** : LLM is better than Regex for detecting violations because it understands context and language nuances, which Regex can't handle well. While both might have similar F1-scores, LLM can adapt to complex conversations and catch mistakes Regex might miss. However, LLM is slower and needs more resources. So, for more accurate results, especially with complicated data, LLM is the best choice.

## Question-3: Visualization Requirements:

In [165]:
def calculate_silence_overtalk_percentage(conversation):
    if len(conversation) == 0:  # Check if the conversation is empty
        return 0, 0  # Return 0% for both silence and overtalk if no conversation data

    total_call_duration = 0  
    overtalk_duration = 0  
    silence_duration = 0 

    # Iterate through the conversation turns
    for i in range(len(conversation) - 1):
        curr_end = conversation[i]['etime']
        next_start = conversation[i + 1]['stime']
        
        # Calculate total call duration
        total_call_duration += conversation[i]['etime'] - conversation[i]['stime']

        # Check for silence (gap between speakers)
        silence = next_start - curr_end
        if silence > 0:
            silence_duration += silence
        
        # Check for overtalk (overlapping speech times)
        curr_start = conversation[i]['stime']
        next_end = conversation[i + 1]['etime']
        if curr_end > next_start and curr_start < next_end:
            overtalk_duration += min(curr_end, next_end) - max(curr_start, next_start)
    
    # Add the last conversation turn duration if the list is not empty
    if len(conversation) > 0:
        total_call_duration += conversation[-1]['etime'] - conversation[-1]['stime']

    # Calculate silence and overtalk percentages
    silence_percentage = (silence_duration / total_call_duration) * 100 if total_call_duration > 0 else 0
    overtalk_percentage = (overtalk_duration / total_call_duration) * 100 if total_call_duration > 0 else 0

    return silence_percentage, overtalk_percentage

# Initialize lists to store results
call_metrics = []

# Iterate through each conversation in the 'conversations' dictionary
for call_id, conversation in conversations.items():
    if conversation:  # Ensure the conversation is not empty
        silence_percentage, overtalk_percentage = calculate_silence_overtalk_percentage(conversation)
        
        # Store results for the current call
        call_metrics.append({
            "call_id": call_id,
            "silence_percentage": silence_percentage,
            "overtalk_percentage": overtalk_percentage
        })
    else:
        print(f"Warning: Conversation for {call_id} is empty.")

# Print the results
for metrics in call_metrics:
    print(f"Call ID: {metrics['call_id']}")
    print(f"Silence Percentage: {metrics['silence_percentage']:.2f}%")
    print(f"Overtalk Percentage: {metrics['overtalk_percentage']:.2f}%")
    print("-" * 50)


Call ID: 2db2965e-54fa-41fa-823b-ed79b943f0b1.json
Silence Percentage: 0.87%
Overtalk Percentage: 13.91%
--------------------------------------------------
Call ID: 8a9655a7-be88-4921-b0ad-04aa1b9953d1.json
Silence Percentage: 8.60%
Overtalk Percentage: 6.45%
--------------------------------------------------
Call ID: 19169ec6-213f-48e9-8ec2-c24ee2e6eb20.json
Silence Percentage: 4.50%
Overtalk Percentage: 10.81%
--------------------------------------------------
Call ID: c86714f0-3aeb-4628-8cfa-ee0a46839508.json
Silence Percentage: 8.14%
Overtalk Percentage: 9.30%
--------------------------------------------------
Call ID: 0c8297aa-ced1-414e-a175-bf29a9763d30.json
Silence Percentage: 7.95%
Overtalk Percentage: 7.95%
--------------------------------------------------
Call ID: 89ab3694-2c42-46bc-a602-c545511bf36e.json
Silence Percentage: 0.53%
Overtalk Percentage: 11.17%
--------------------------------------------------
Call ID: 94ecf497-5959-44ec-a011-92af3709c38c.json
Silence Percenta